# 🔍 Pesquisa Completa no Gov.br por CPF

Este notebook pesquisa informações públicas relacionadas a um CPF em múltiplas APIs do Portal da Transparência.

## ⚠️ Importante
- **Dados retornados são públicos e agregados**
- **Não retorna dados pessoais sensíveis** (nome completo, endereço, email)
- **Apenas informações de transparência pública** (benefícios, servidores, empresas)
- **Respeita LGPD e privacidade dos cidadãos**

## 1. Configuração e Imports

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import json
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Configurações
PORTAL_TRANSPARENCIA_API_KEY = "2c56919ba91b8c1b13473dcef43fb031"
transparency_url = "https://api.portaldatransparencia.gov.br/api-de-dados"

headers = {
    'chave-api-dados': PORTAL_TRANSPARENCIA_API_KEY,
    'Accept': 'application/json'
}

print("✅ Configurações carregadas!")

## 2. Funções Auxiliares

In [ ]:
def formatar_cpf(cpf):
    """Remove formatação do CPF"""
    return ''.join(filter(str.isdigit, str(cpf)))

def mascarar_cpf(cpf):
    """Mascara CPF para exibição (XXX.XXX.XXX-XX)"""
    cpf_limpo = formatar_cpf(cpf)
    if len(cpf_limpo) == 11:
        return f"{cpf_limpo[:3]}.{cpf_limpo[3:6]}.{cpf_limpo[6:9]}-{cpf_limpo[9:]}"
    return cpf_limpo

def consultar_endpoint(endpoint, params, descricao):
    """Consulta um endpoint da API e retorna os dados"""
    try:
        response = requests.get(
            f"{transparency_url}{endpoint}",
            headers=headers,
            params=params,
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            if data and len(data) > 0:
                return {
                    'status': 'success',
                    'data': data,
                    'count': len(data) if isinstance(data, list) else 1
                }
            else:
                return {
                    'status': 'empty',
                    'data': [],
                    'count': 0
                }
        elif response.status_code == 401:
            return {
                'status': 'unauthorized',
                'data': [],
                'count': 0,
                'error': 'Não autorizado - verifique a chave API'
            }
        else:
            return {
                'status': 'error',
                'data': [],
                'count': 0,
                'error': f"Status {response.status_code}: {response.text[:200]}"
            }
    except Exception as e:
        return {
            'status': 'error',
            'data': [],
            'count': 0,
            'error': str(e)
        }

print("✅ Funções auxiliares criadas!")

## 3. Entrada do CPF

In [ ]:
# INSIRA O CPF AQUI (com ou sem formatação)
CPF_INPUT = ""  # Exemplo: "12345678901" ou "123.456.789-01"

# Se vazio, solicitar input
if not CPF_INPUT:
    CPF_INPUT = input("Digite o CPF para pesquisa (com ou sem formatação): ")

CPF = formatar_cpf(CPF_INPUT)

if len(CPF) != 11:
    print(f"❌ CPF inválido! Deve ter 11 dígitos. Recebido: {len(CPF)} dígitos")
    raise ValueError("CPF inválido")

CPF_MASCARADO = mascarar_cpf(CPF)
print(f"\n🔍 Pesquisando informações para CPF: {CPF_MASCARADO}")
print("=" * 80)

## 4. Pesquisa em Múltiplos Endpoints

In [ ]:
resultados = {}

print("\n📊 Iniciando pesquisa em múltiplos endpoints...\n")

# 1. Bolsa Família por CPF/NIS
print("[1/7] Consultando Bolsa Família por CPF/NIS...")
meses_para_testar = 12  # Últimos 12 meses
bolsa_familia_resultados = []

for i in range(meses_para_testar):
    data_ref = (datetime.now() - timedelta(days=30*i)).strftime('%Y%m')
    result = consultar_endpoint(
        '/bolsa-familia-por-cpf-ou-nis',
        {'cpfOuNis': CPF, 'mesAno': data_ref, 'pagina': 1},
        f'Bolsa Família - {data_ref}'
    )
    if result['status'] == 'success' and result['count'] > 0:
        bolsa_familia_resultados.extend(result['data'])

resultados['bolsa_familia'] = {
    'status': 'success' if bolsa_familia_resultados else 'empty',
    'data': bolsa_familia_resultados,
    'count': len(bolsa_familia_resultados)
}
print(f"   {'✅' if bolsa_familia_resultados else '⚠️ '} {len(bolsa_familia_resultados)} registro(s) encontrado(s)")

# 2. Servidores Públicos
print("\n[2/7] Consultando Servidores Públicos...")
mes_atual = datetime.now().strftime('%Y%m')
result_servidores = consultar_endpoint(
    '/servidores',
    {'cpf': CPF, 'mesAnoReferencia': mes_atual, 'pagina': 1},
    'Servidores Públicos'
)
resultados['servidores'] = result_servidores
print(f"   {'✅' if result_servidores['count'] > 0 else '⚠️ '} {result_servidores['count']} registro(s) encontrado(s)")

# 3. CEIS - Cadastro de Empresas Inidôneas e Suspensas
print("\n[3/7] Consultando CEIS (Empresas Inidôneas)...")
result_ceis = consultar_endpoint(
    '/ceis',
    {'cpfOuCnpj': CPF, 'pagina': 1},
    'CEIS'
)
resultados['ceis'] = result_ceis
print(f"   {'✅' if result_ceis['count'] > 0 else '⚠️ '} {result_ceis['count']} registro(s) encontrado(s)")

# 4. CNEP - Cadastro Nacional de Empresas Punidas
print("\n[4/7] Consultando CNEP (Empresas Punidas)...")
result_cnep = consultar_endpoint(
    '/cnep',
    {'cpfOuCnpj': CPF, 'pagina': 1},
    'CNEP'
)
resultados['cnep'] = result_cnep
print(f"   {'✅' if result_cnep['count'] > 0 else '⚠️ '} {result_cnep['count']} registro(s) encontrado(s)")

# 5. Despesas (se o CPF for fornecedor)
print("\n[5/7] Consultando Despesas Públicas...")
mes_inicio = (datetime.now() - timedelta(days=365)).strftime('%Y%m')
mes_fim = datetime.now().strftime('%Y%m')
result_despesas = consultar_endpoint(
    '/despesas/documentos',
    {'cpfCnpjFornecedor': CPF, 'mesAnoInicio': mes_inicio, 'mesAnoFim': mes_fim, 'pagina': 1},
    'Despesas Públicas'
)
resultados['despesas'] = result_despesas
print(f"   {'✅' if result_despesas['count'] > 0 else '⚠️ '} {result_despesas['count']} registro(s) encontrado(s)")

# 6. Convênios (se o CPF for proponente/executor)
print("\n[6/7] Consultando Convênios...")
data_inicio = (datetime.now() - timedelta(days=365)).strftime('%d/%m/%Y')
data_fim = datetime.now().strftime('%d/%m/%Y')
result_convenios = consultar_endpoint(
    '/convenios',
    {'cpfCnpjProponente': CPF, 'dataInicialCelebracao': data_inicio, 'dataFinalCelebracao': data_fim, 'pagina': 1},
    'Convênios'
)
resultados['convenios'] = result_convenios
print(f"   {'✅' if result_convenios['count'] > 0 else '⚠️ '} {result_convenios['count']} registro(s) encontrado(s)")

# 7. Contratos
print("\n[7/7] Consultando Contratos...")
data_inicio_contrato = (datetime.now() - timedelta(days=365)).strftime('%d/%m/%Y')
data_fim_contrato = datetime.now().strftime('%d/%m/%Y')
result_contratos = consultar_endpoint(
    '/contratos',
    {'cpfCnpjContratado': CPF, 'dataInicial': data_inicio_contrato, 'dataFinal': data_fim_contrato, 'pagina': 1},
    'Contratos'
)
resultados['contratos'] = result_contratos
print(f"   {'✅' if result_contratos['count'] > 0 else '⚠️ '} {result_contratos['count']} registro(s) encontrado(s)")

print("\n" + "=" * 80)
print("✅ Pesquisa concluída!")
print("=" * 80)

## 5. Resumo dos Resultados

In [ ]:
print(f"\n📊 RESUMO DA PESQUISA - CPF: {CPF_MASCARADO}\n")
print("=" * 80)

resumo = []
total_registros = 0

for categoria, resultado in resultados.items():
    count = resultado.get('count', 0)
    status_emoji = '✅' if count > 0 else '⚠️'
    status_texto = 'Encontrado' if count > 0 else 'Não encontrado'
    
    resumo.append({
        'Categoria': categoria.replace('_', ' ').title(),
        'Status': status_texto,
        'Registros': count
    })
    total_registros += count

df_resumo = pd.DataFrame(resumo)
display(df_resumo)

print(f"\n📈 Total de registros encontrados: {total_registros}")

if total_registros == 0:
    print("\n⚠️  Nenhum registro encontrado. Possíveis motivos:")
    print("   - CPF não possui registros públicos nos sistemas consultados")
    print("   - Período de consulta não contém dados")
    print("   - Dados podem não estar disponíveis na API")
else:
    print("\n✅ Dados encontrados! Veja os detalhes abaixo.")

## 6. Detalhamento por Categoria

In [ ]:
# Função para exibir dados de forma organizada
def exibir_resultados(categoria, dados, titulo):
    if not dados or len(dados) == 0:
        print(f"\n⚠️  {titulo}: Nenhum registro encontrado")
        return
    
    print(f"\n{'='*80}")
    print(f"📋 {titulo}")
    print(f"{'='*80}")
    
    try:
        df = pd.DataFrame(dados)
        print(f"\nTotal de registros: {len(df)}")
        print(f"\nColunas disponíveis: {', '.join(df.columns.tolist())}")
        print("\n" + "-"*80)
        display(df.head(20))  # Mostrar primeiros 20 registros
        
        if len(df) > 20:
            print(f"\n... e mais {len(df) - 20} registro(s)")
    except Exception as e:
        print(f"\n⚠️  Erro ao processar dados: {e}")
        print(f"\nDados brutos (JSON):")
        print(json.dumps(dados[:3], indent=2, ensure_ascii=False))

# Exibir cada categoria
if resultados['bolsa_familia']['count'] > 0:
    exibir_resultados('bolsa_familia', resultados['bolsa_familia']['data'], 'Bolsa Família')

if resultados['servidores']['count'] > 0:
    exibir_resultados('servidores', resultados['servidores']['data'], 'Servidores Públicos')

if resultados['ceis']['count'] > 0:
    exibir_resultados('ceis', resultados['ceis']['data'], 'CEIS - Empresas Inidôneas')

if resultados['cnep']['count'] > 0:
    exibir_resultados('cnep', resultados['cnep']['data'], 'CNEP - Empresas Punidas')

if resultados['despesas']['count'] > 0:
    exibir_resultados('despesas', resultados['despesas']['data'], 'Despesas Públicas')

if resultados['convenios']['count'] > 0:
    exibir_resultados('convenios', resultados['convenios']['data'], 'Convênios')

if resultados['contratos']['count'] > 0:
    exibir_resultados('contratos', resultados['contratos']['data'], 'Contratos')

## 7. Exportar Resultados

In [ ]:
# Exportar para JSON
exportar_json = True  # Mude para False se não quiser exportar

if exportar_json and total_registros > 0:
    arquivo_json = f"pesquisa_cpf_{CPF}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    dados_export = {
        'cpf': CPF_MASCARADO,
        'data_pesquisa': datetime.now().isoformat(),
        'total_registros': total_registros,
        'resultados': {}
    }
    
    for categoria, resultado in resultados.items():
        if resultado['count'] > 0:
            dados_export['resultados'][categoria] = resultado['data']
    
    with open(arquivo_json, 'w', encoding='utf-8') as f:
        json.dump(dados_export, f, indent=2, ensure_ascii=False, default=str)
    
    print(f"\n✅ Resultados exportados para: {arquivo_json}")
else:
    print("\n💡 Para exportar resultados, altere 'exportar_json' para True")

## 8. Observações Importantes

### ⚠️ Limitações e Considerações

1. **Dados Pessoais**: As APIs não retornam dados pessoais completos (nome completo, endereço, email)
2. **Privacidade**: Apenas informações públicas de transparência são retornadas
3. **LGPD**: Todos os dados respeitam a Lei Geral de Proteção de Dados
4. **Período**: Algumas consultas limitam o período histórico disponível
5. **Rate Limiting**: A API pode ter limites de requisições por minuto

### 📋 O que é retornado:
- **Benefícios**: Valores e períodos de benefícios sociais
- **Servidores**: Informações públicas sobre servidores (se aplicável)
- **Empresas**: Relação com empresas inidôneas ou punidas
- **Contratos/Convênios**: Relação com o poder público
- **Despesas**: Pagamentos recebidos do governo

### ❌ O que NÃO é retornado:
- Nome completo do titular
- Endereço residencial
- Email pessoal
- Telefone
- Outros dados pessoais sensíveis

### 📚 Documentação:
- Portal da Transparência: https://portaldatransparencia.gov.br/api-de-dados
- Swagger/API Docs: https://portaldatransparencia.gov.br/api-de-dados/swagger-ui.html